### Lab3 - Kompresja macierzy (Jan Masternak, Jakub Mróz):

#### Kwestie techniczne:

* Pomiary wykonywane na procesorze i5-13400f będą miały przed bezpośrednio przed rozszerzeniem końcówkę "...i5_13400f"

##### Importy potrzebnych paczek:

Poniższe wystarczy uruchomić jeden raz.

In [ ]:
using Pkg
# Pkg.add("CSV")
# Pkg.add("DataFrames")
# Pkg.add("Plots")
# Pkg.add("LinearAlgebra")
# Pkg.add("LaTeXStrings")
# Pkg.add("StatsPlots")
# Pkg.add("Images")
# Pkg.add("ImageIO")
# Pkg.add("ColorTypes")
# Pkg.add("ImageView")
# Pkg.add("Printf")
# Pkg.add("WebP")
# Pkg.add("Statistics");

Poniższe uruchamiamy każdorazowo.

In [ ]:
using CSV
using DataFrames
using Plots
using LinearAlgebra
using LaTeXStrings
using StatsPlots
using Images
using ImageIO
using ColorTypes
using ImageView
using Printf
using Statistics;

In [ ]:
cpu = "i5_13400f"
error_eps = 1e-8;

In [ ]:
function generate_random_matrix(m, n, a, b)
    A = zeros(m, n)
    for i in 1:1:m 
        for j in 1:1:n 
            A[i,j] = a + (b-a)*rand()
        end
    end
    return A
end

function are_equal(A, B)
    if !all(size(A) .== size(B))
        return false
    end

    return all(abs.(A-B) .< error_eps)

end

function pad(A, target_rows, target_cols)
    rows, cols = A.size()
    return [A zeros(rows, target_cols-cols) ; zeros(target_rows-rows, target_cols)]
end;

##### Ładowanie bitmap:

In [ ]:
loaded_bitmap = "bitmaps/golden_pheasant.png";

In [ ]:
abstract1 = load(loaded_bitmap);

##### Display bitmap:

In [ ]:
abstract1

##### Tworzenie tablic RGB dla bitmap:

In [ ]:
function from_1_to_255_RGB_repr(x::Float64)
    return x*255
end

function from_255_to_1_RGB_repr(x::Float64)
    return x/255
end;

In [ ]:
abstract1_rows, abstract1_cols = size(abstract1)
abstract1R, abstract1G, abstract1B = zeros(abstract1_rows, abstract1_cols), zeros(abstract1_rows, abstract1_cols), zeros(abstract1_rows, abstract1_cols)

for i in 1:1:abstract1_rows
    for j in 1:1:abstract1_cols
        pixel_ij = abstract1[i,j]
        abstract1R[i,j] = red(pixel_ij)
        abstract1G[i,j] = green(pixel_ij)
        abstract1B[i,j] = blue(pixel_ij)
    end
end

##### Sprawdzenie poprawności działania tworzenia tablic RGB:

In [ ]:
built_bitmap = "bitmaps/built_golden_pheasant.png"
built_abstract1 = colorview(RGB, abstract1R, abstract1G, abstract1B)
save(built_bitmap, built_abstract1)
img_built_abstract1 = load(built_bitmap)

### Tworzenie macierzy kompresji z użyciem truncated SVD:

##### Konwersja macierzy R, G, B z przedziału [0, 1] na [0, 255]:

In [ ]:
abstract1R_255, abstract1G_255, abstract1B_255 = deepcopy(abstract1R), deepcopy(abstract1G), deepcopy(abstract1B)
abstract1R_255 = from_1_to_255_RGB_repr.(abstract1R_255)
abstract1G_255 = from_1_to_255_RGB_repr.(abstract1G_255)
abstract1B_255 = from_1_to_255_RGB_repr.(abstract1B_255);

##### Kompresja macierzy z użyciem truncated SVD:

In [ ]:
module CompressionTreeModule

mutable struct CompressionTreeNode
    rank
    addr

    left_upper_child
    right_upper_child
    left_lower_child
    right_lower_child

    singular_values
    U_matrix
    V_tr_matrix
end

end;

using .CompressionTreeModule;

function create_empty_compr_tree_node()
    return CompressionTreeModule.CompressionTreeNode(nothing, nothing, 
                               nothing, nothing, nothing, nothing,
                               nothing, nothing, nothing)
end;

In [ ]:
function compress_matrix(A, r_min, r_max, c_min, c_max, U, D, V_tr, gamma)
    if all(abs.(A[r_min:r_max , c_min:c_max]) .< error_eps)
        v = create_empty_compr_tree_node()
        v.rank = 0
        v.addr = (r_min, r_max, c_min, c_max)
        return v
    else
        sigmas = diag(D)
        v = create_empty_compr_tree_node()
        v.rank = gamma
        v.addr = (r_min, r_max, c_min, c_max)
        v.singular_values = sigmas[1:gamma]
        v.U_matrix = U[: , 1:gamma]
        v.V_tr_matrix = D[1:gamma , 1:gamma] * V_tr[1:gamma , :]
        return v
    end
end;

In [ ]:
function trunc_svd(A; gamma=min(size(A,1), size(A,2)))
    gamma = min(gamma, min(size(A,1), size(A,2)))
    U, D, V = svd(A)
    D = diagm(D)
    U = U[: , 1:gamma]
    D = D[1:gamma , 1:gamma]
    V = V[: , 1:gamma]
    return U, D, V'
end;

In [ ]:
function create_tree(A, r_min=1, r_max=size(A,1), c_min=1, c_max=size(A,2); gamma=1, eps=1)
    if r_min == r_max && c_min == c_max
        v = create_empty_compr_tree_node()
        v.rank = 1
        v.addr = (r_min, r_max, c_min, c_max)
        v.U_matrix = reshape([A[r_min, c_min]], 1, 1)
        return v
    end   
    
    U, D, V_tr = trunc_svd(A[r_min:r_max , c_min:c_max], gamma=gamma+1)
    if gamma+1 > size(D,1)
        gamma = size(D,1)
        if D[gamma, gamma] - eps < error_eps
            v = compress_matrix(A, r_min, r_max, c_min, c_max, U, D, V_tr, gamma)
            return v 
        end
    elseif D[gamma+1, gamma+1] - eps < error_eps
        v = compress_matrix(A, r_min, r_max, c_min, c_max, U, D, V_tr, gamma)
        return v
    end 

    r_mid = div(r_min+r_max,2)
    c_mid = div(c_min+c_max,2)
    right_upper_child, left_lower_child, right_lower_child = nothing, nothing, nothing
    left_upper_child = create_tree(A, r_min, r_mid, c_min, c_mid, gamma=gamma, eps=eps)
    if c_mid < c_max
        right_upper_child = create_tree(A, r_min, r_mid, c_mid+1, c_max, gamma=gamma, eps=eps)
    end
    if r_mid < r_max
        left_lower_child = create_tree(A, r_mid+1, r_max, c_min, c_mid, gamma=gamma, eps=eps)
        if c_mid < c_max
            right_lower_child = create_tree(A, r_mid+1, r_max, c_mid+1, c_max, gamma=gamma, eps=eps)
        end            
    end

    v = create_empty_compr_tree_node()
    v.addr = (r_min, r_max, c_min, c_max)
    v.left_upper_child = left_upper_child
    v.right_upper_child = right_upper_child
    v.left_lower_child = left_lower_child
    v.right_lower_child = right_lower_child
    return v

end;

##### Testowanie kompresji macierzy z TSVD:

Poniżej przetestowano poprawne działanie zaimplementowanego algorytmu dla losowych danych - ta sekcja jeszcze nie ma nic stricte wspólnego z kompresją bitampy.

In [ ]:
function build_matrix_based_on_tree(target_matrix, node)
    r_min, r_max, c_min, c_max = node.addr
    
    if !isnothing(node.U_matrix)
        if !isnothing(node.V_tr_matrix)
            target_matrix[r_min:r_max , c_min:c_max] += node.U_matrix*node.V_tr_matrix
        else
            target_matrix[r_min:r_max , c_min:c_max] += node.U_matrix;
        end
    end

    if !isnothing(node.left_upper_child)
        build_matrix_based_on_tree(target_matrix, node.left_upper_child)
    end

    if !isnothing(node.right_upper_child)
        build_matrix_based_on_tree(target_matrix, node.right_upper_child)
    end
    
    if !isnothing(node.left_lower_child)
        build_matrix_based_on_tree(target_matrix, node.left_lower_child)
    end
    
    if !isnothing(node.right_lower_child)
        build_matrix_based_on_tree(target_matrix, node.right_lower_child)
    end
    
end;

In [ ]:
function test_matrix_compression(low_m, high_m, low_n, high_n, a, b, tests, gamma, eps)
    correct_outputs = 0
    incorrect_ouputs = 0
    for _ in 1:1:tests 
        A = generate_random_matrix(rand(low_m:1:high_m), rand(low_n:1:high_n), a, b)
        tree_root = create_tree(A, gamma=gamma, eps=eps)
        built_A = zeros(size(A,1), size(A,2))
        build_matrix_based_on_tree(built_A, tree_root)
        if are_equal(A, built_A)
            correct_outputs += 1
        else 
            incorrect_ouputs += 1    
        end
    end

    return correct_outputs, incorrect_ouputs
end;

In [ ]:
low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test = 1, 10, 1, 10
a_for_test, b_for_test = 0, 50
correct, incorrect = test_matrix_compression(low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test, a_for_test, b_for_test, 15, 1, 0.1)
println(correct, " ", incorrect);

I dla nieco większego $\epsilon$:

In [ ]:
low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test = 1, 10, 1, 10
a_for_test, b_for_test = 0, 50
correct, incorrect = test_matrix_compression(low_m_for_test, high_m_for_test, low_n_for_test, high_n_for_test, a_for_test, b_for_test, 15, 1, 6)
println(correct, " ", incorrect);

### Rezultaty kompresji dla badanej bitmapy:

W tej sekcji zbadano jak dobre bitmapy można uzyskać po skompresowaniu, a następnie zbudowanu ich na nowo dla różnych argumentów algorytmu kompresji.

In [ ]:
_, eigenvaluesR, _ = svd(abstract1R_255)
_, eigenvaluesG, _ = svd(abstract1G_255)
_, eigenvaluesB, _ = svd(abstract1B_255);

Narysujmy wartości własne tych macierzy:

In [ ]:
scatter(eigenvaluesR, [1 for i in 1:1:size(eigenvaluesR,1)], label="Eigenvalues dla R", color=:red)
scatter!(eigenvaluesG, [2 for i in 1:1:size(eigenvaluesG,1)], label="Eigenvalues dla G", color=:green)
scatter!(eigenvaluesB, [3 for i in 1:1:size(eigenvaluesB,1)], label="Eigenvalues dla B", color=:blue)

Eigenvalues nie licząc bardzo dużej $\sigma_{1}$:

In [ ]:
scatter(eigenvaluesR[2:end], [1 for i in 1:1:size(eigenvaluesR,1)-1], label="Eigenvalues dla R", color=:red)
scatter!(eigenvaluesG[2:end], [2 for i in 1:1:size(eigenvaluesG,1)-1], label="Eigenvalues dla G", color=:green)
scatter!(eigenvaluesB[2:end], [3 for i in 1:1:size(eigenvaluesB,1)-1], label="Eigenvalues dla B", color=:blue)

Jeszcze dokładniej:

In [ ]:
scatter(eigenvaluesR[80:end], [1 for i in 1:1:size(eigenvaluesR,1)-79], label="Eigenvalues dla R", color=:red)
scatter!(eigenvaluesG[80:end], [2 for i in 1:1:size(eigenvaluesG,1)-79], label="Eigenvalues dla G", color=:green)
scatter!(eigenvaluesB[80:end], [3 for i in 1:1:size(eigenvaluesB,1)-79], label="Eigenvalues dla B", color=:blue)

In [ ]:
slight_increase = 1e-2

sigmaR_1 = eigenvaluesR[end]+slight_increase
sigmaR_n = eigenvaluesR[2]+slight_increase
sigmaR_ndiv2 = eigenvaluesR[div(size(eigenvaluesR,1),2)]+slight_increase

sigmaG_1 = eigenvaluesG[end]+slight_increase
sigmaG_n = eigenvaluesG[2]+slight_increase
sigmaG_ndiv2 = eigenvaluesG[div(size(eigenvaluesG,1),2)]+slight_increase

sigmaB_1 = eigenvaluesB[end]+slight_increase
sigmaB_n = eigenvaluesB[2]+slight_increase
sigmaB_ndiv2 = eigenvaluesB[div(size(eigenvaluesB,1),2)]+slight_increase

execution_argsR = [
    Dict("gamma" => 1, "eps" => sigmaR_1),
    Dict("gamma" => 1, "eps" => sigmaR_n),
    Dict("gamma" => 1, "eps" => sigmaR_ndiv2),
    Dict("gamma" => 4, "eps" => sigmaR_1),
    Dict("gamma" => 4, "eps" => sigmaR_n),
    Dict("gamma" => 4, "eps" => sigmaR_ndiv2),
    Dict("gamma" => size(eigenvaluesR,1)-1, "eps" => sigmaR_1)
]

execution_argsG = [
    Dict("gamma" => 1, "eps" => sigmaG_1),
    Dict("gamma" => 1, "eps" => sigmaG_n),
    Dict("gamma" => 1, "eps" => sigmaG_ndiv2),
    Dict("gamma" => 4, "eps" => sigmaG_1),
    Dict("gamma" => 4, "eps" => sigmaG_n),
    Dict("gamma" => 4, "eps" => sigmaG_ndiv2),
    Dict("gamma" => size(eigenvaluesG,1)-1, "eps" => sigmaG_1)
]

execution_argsB = [
    Dict("gamma" => 1, "eps" => sigmaB_1),
    Dict("gamma" => 1, "eps" => sigmaB_n),
    Dict("gamma" => 1, "eps" => sigmaB_ndiv2),
    Dict("gamma" => 4, "eps" => sigmaB_1),
    Dict("gamma" => 4, "eps" => sigmaB_n),
    Dict("gamma" => 4, "eps" => sigmaB_ndiv2),
    Dict("gamma" => size(eigenvaluesB,1)-1, "eps" => sigmaB_1)
];

In [ ]:
built_trees = Vector{Tuple{Any, Any, Any}}(undef, size(execution_argsR,1))

for i in 1:1:size(execution_argsR,1)
    red_gamma, red_eps = execution_argsR[i]["gamma"], execution_argsR[i]["eps"]
    green_gamma, green_eps = execution_argsG[i]["gamma"], execution_argsG[i]["eps"]
    blue_gamma, blue_eps = execution_argsB[i]["gamma"], execution_argsB[i]["eps"]

    red_root = create_tree(abstract1R_255, gamma=red_gamma, eps=red_eps)
    green_root = create_tree(abstract1G_255, gamma=green_gamma, eps=green_eps)
    blue_root = create_tree(abstract1B_255, gamma=blue_gamma, eps=blue_eps)

    built_trees[i] = (red_root, green_root, blue_root)
end;

Poniżej zaimplementowano funkcje pozwalają wygodnie porównać bitmapy przed i to kompresji:

In [ ]:
function create_image_comparing_bitmaps(true_bitmap, built_bitmap, filename; whitespace_between=25) 
    if !all(size(true_bitmap) .== size(built_bitmap))
        println("Sizes are incompatible")
    else
        height = size(true_bitmap, 1)
        comparison_matrix = [true_bitmap ones(height, whitespace_between) built_bitmap]
        save(filename, comparison_matrix)
    end
end;

In [ ]:
function create_image_comparing_RGB_bitmaps(true_R, true_G, true_B, built_R, built_G, built_B, filename; whitespace_between=25)
    if !all(size(true_R) .== size(built_R))
        println("Sizes are incompatible")
    elseif !all(size(true_G) .== size(built_G))
        println("Sizes are incompatible")    
    elseif !all(size(true_B) .== size(built_B))
        println("Sizes are incompatible")    
    else
        height = size(true_R, 1)
        compared_R = [true_R ones(height, whitespace_between) built_R]
        compared_G = [true_G ones(height, whitespace_between) built_G]
        compared_B = [true_B ones(height, whitespace_between) built_B]
        compared_RGB = colorview(RGB, compared_R, compared_G, compared_B)
        save(filename, compared_RGB)
    end
end;

##### Porównania dla $\gamma = 1$, $\epsilon = \sigma_{1}$:

In [ ]:
w, h = size(abstract1R_255);

In [ ]:
index = 1
built_R = zeros(w,h)
built_G = zeros(w,h)
built_B = zeros(w,h)
red_root, green_root, blue_root = built_trees[index]

build_matrix_based_on_tree(built_R, red_root)
build_matrix_based_on_tree(built_G, green_root)
build_matrix_based_on_tree(built_B, blue_root)

In [ ]:
built_R = built_R./255
built_G = built_G./255
built_B = built_B./255;

built_R = min.(built_R, 1)
built_R = max.(built_R, 0)

built_G = min.(built_G, 1)
built_G = max.(built_G, 0)

built_B = min.(built_B, 1)
built_B = max.(built_B, 0);

Porównanie czerwonego (R):

In [ ]:
red_1_filename = "bitmaps/red_1_comparison.png"
create_image_comparing_bitmaps(abstract1R, built_R, red_1_filename, whitespace_between=40)
load(red_1_filename)

Porównanie zielonego (G):

In [ ]:
green_1_filename = "bitmaps/green_1_comparison.png"
create_image_comparing_bitmaps(abstract1G, built_G, green_1_filename, whitespace_between=40)
load(green_1_filename)

Porównanie niebieskiego (B):

In [ ]:
blue_1_filename = "bitmaps/blue_1_comparison.png"
create_image_comparing_bitmaps(abstract1B, built_B, blue_1_filename, whitespace_between=40)
load(blue_1_filename)

Porównanie całości:

In [ ]:
overall_1_filename = "bitmaps/overall_1_comparison.png"
create_image_comparing_RGB_bitmaps(abstract1R, abstract1G, abstract1B, built_R, built_G, built_B, overall_1_filename, whitespace_between=40)
load(overall_1_filename)

##### Porównanie dla $\gamma = 1$ i $\epsilon = \sigma_{n-1}$:

In [ ]:
index = 2
built_R = zeros(w,h)
built_G = zeros(w,h)
built_B = zeros(w,h)
red_root, green_root, blue_root = built_trees[index]

build_matrix_based_on_tree(built_R, red_root)
build_matrix_based_on_tree(built_G, green_root)
build_matrix_based_on_tree(built_B, blue_root)

In [ ]:
built_R = built_R./255
built_G = built_G./255
built_B = built_B./255;

built_R = min.(built_R, 1)
built_R = max.(built_R, 0)

built_G = min.(built_G, 1)
built_G = max.(built_G, 0)

built_B = min.(built_B, 1)
built_B = max.(built_B, 0);

Porównanie czerwonego (R):

In [ ]:
red_filename = @sprintf("bitmaps/red_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1R, built_R, red_filename, whitespace_between=40)
load(red_filename)

Porównanie zielonego (G):

In [ ]:
green_filename = @sprintf("bitmaps/green_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1G, built_G, green_filename, whitespace_between=40)
load(green_filename)

Porówananie niebieskiego (B):

In [ ]:
blue_filename = @sprintf("bitmaps/blue_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1B, built_B, blue_filename, whitespace_between=40)
load(blue_filename)

Porównanie całości:

In [ ]:
overall_filename = @sprintf("bitmaps/overall_%d_comparison.png", index)
create_image_comparing_RGB_bitmaps(abstract1R, abstract1G, abstract1B, built_R, built_G, built_B, overall_filename, whitespace_between=40)
load(overall_filename)

##### Porównanie dla $\gamma = 1$ i $\epsilon = \sigma_{\lfloor \frac{n}{2} \rfloor}$:

In [ ]:
index = 3
built_R = zeros(w,h)
built_G = zeros(w,h)
built_B = zeros(w,h)
red_root, green_root, blue_root = built_trees[index]

build_matrix_based_on_tree(built_R, red_root)
build_matrix_based_on_tree(built_G, green_root)
build_matrix_based_on_tree(built_B, blue_root)

In [ ]:
built_R = built_R./255
built_G = built_G./255
built_B = built_B./255;

built_R = min.(built_R, 1)
built_R = max.(built_R, 0)

built_G = min.(built_G, 1)
built_G = max.(built_G, 0)

built_B = min.(built_B, 1)
built_B = max.(built_B, 0);

Porównanie czerwonego (R):

In [ ]:
red_filename = @sprintf("bitmaps/red_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1R, built_R, red_filename, whitespace_between=40)
load(red_filename)

Porównanie zielonego (G):

In [ ]:
green_filename = @sprintf("bitmaps/green_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1G, built_G, green_filename, whitespace_between=40)
load(green_filename)

Porówananie niebieskiego (B):

In [ ]:
blue_filename = @sprintf("bitmaps/blue_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1B, built_B, blue_filename, whitespace_between=40)
load(blue_filename)

Porównanie całości:

In [ ]:
overall_filename = @sprintf("bitmaps/overall_%d_comparison.png", index)
create_image_comparing_RGB_bitmaps(abstract1R, abstract1G, abstract1B, built_R, built_G, built_B, overall_filename, whitespace_between=40)
load(overall_filename)

##### Porównanie dla $\gamma = 4$ i $\epsilon = \sigma_{1}$:

In [ ]:
index = 4
built_R = zeros(w,h)
built_G = zeros(w,h)
built_B = zeros(w,h)
red_root, green_root, blue_root = built_trees[index]

build_matrix_based_on_tree(built_R, red_root)
build_matrix_based_on_tree(built_G, green_root)
build_matrix_based_on_tree(built_B, blue_root)

In [ ]:
built_R = built_R./255
built_G = built_G./255
built_B = built_B./255;

built_R = min.(built_R, 1)
built_R = max.(built_R, 0)

built_G = min.(built_G, 1)
built_G = max.(built_G, 0)

built_B = min.(built_B, 1)
built_B = max.(built_B, 0);

Porównanie czerwonego (R):

In [ ]:
red_filename = @sprintf("bitmaps/red_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1R, built_R, red_filename, whitespace_between=40)
load(red_filename)

Porównanie zielonego (G):

In [ ]:
green_filename = @sprintf("bitmaps/green_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1G, built_G, green_filename, whitespace_between=40)
load(green_filename)

Porówananie niebieskiego (B):

In [ ]:
blue_filename = @sprintf("bitmaps/blue_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1B, built_B, blue_filename, whitespace_between=40)
load(blue_filename)

Porównanie całości:

In [ ]:
overall_filename = @sprintf("bitmaps/overall_%d_comparison.png", index)
create_image_comparing_RGB_bitmaps(abstract1R, abstract1G, abstract1B, built_R, built_G, built_B, overall_filename, whitespace_between=40)
load(overall_filename)

##### Porównanie dla $\gamma = 4$ i $\epsilon = \sigma_{n}$:

In [ ]:
index = 5
built_R = zeros(w,h)
built_G = zeros(w,h)
built_B = zeros(w,h)
red_root, green_root, blue_root = built_trees[index]

build_matrix_based_on_tree(built_R, red_root)
build_matrix_based_on_tree(built_G, green_root)
build_matrix_based_on_tree(built_B, blue_root)

In [ ]:
built_R = built_R./255
built_G = built_G./255
built_B = built_B./255;

built_R = min.(built_R, 1)
built_R = max.(built_R, 0)

built_G = min.(built_G, 1)
built_G = max.(built_G, 0)

built_B = min.(built_B, 1)
built_B = max.(built_B, 0);

Porównanie czerwonego (R):

In [ ]:
red_filename = @sprintf("bitmaps/red_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1R, built_R, red_filename, whitespace_between=40)
load(red_filename)

Porównanie zielonego (G):

In [ ]:
green_filename = @sprintf("bitmaps/green_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1G, built_G, green_filename, whitespace_between=40)
load(green_filename)

Porówananie niebieskiego (B):

In [ ]:
blue_filename = @sprintf("bitmaps/blue_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1B, built_B, blue_filename, whitespace_between=40)
load(blue_filename)

Porównanie całości:

In [ ]:
overall_filename = @sprintf("bitmaps/overall_%d_comparison.png", index)
create_image_comparing_RGB_bitmaps(abstract1R, abstract1G, abstract1B, built_R, built_G, built_B, overall_filename, whitespace_between=40)
load(overall_filename)

##### Porównanie dla $\gamma = 4$ i $\epsilon = \sigma_{\lfloor \frac{n}{2} \rfloor }$:

In [ ]:
index = 6
built_R = zeros(w,h)
built_G = zeros(w,h)
built_B = zeros(w,h)
red_root, green_root, blue_root = built_trees[index]

build_matrix_based_on_tree(built_R, red_root)
build_matrix_based_on_tree(built_G, green_root)
build_matrix_based_on_tree(built_B, blue_root)

In [ ]:
built_R = built_R./255
built_G = built_G./255
built_B = built_B./255;

built_R = min.(built_R, 1)
built_R = max.(built_R, 0)

built_G = min.(built_G, 1)
built_G = max.(built_G, 0)

built_B = min.(built_B, 1)
built_B = max.(built_B, 0);

Porównanie czerwonego (R):

In [ ]:
red_filename = @sprintf("bitmaps/red_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1R, built_R, red_filename, whitespace_between=40)
load(red_filename)

Porównanie zielonego (G):

In [ ]:
green_filename = @sprintf("bitmaps/green_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1G, built_G, green_filename, whitespace_between=40)
load(green_filename)

Porówananie niebieskiego (B):

In [ ]:
blue_filename = @sprintf("bitmaps/blue_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1B, built_B, blue_filename, whitespace_between=40)
load(blue_filename)

Porównanie całości:

In [ ]:
overall_filename = @sprintf("bitmaps/overall_%d_comparison.png", index)
create_image_comparing_RGB_bitmaps(abstract1R, abstract1G, abstract1B, built_R, built_G, built_B, overall_filename, whitespace_between=40)
load(overall_filename)

##### Porównanie dla $\gamma = n-1$ i $\epsilon = \sigma_{1}$:

In [ ]:
index = 7
built_R = zeros(w,h)
built_G = zeros(w,h)
built_B = zeros(w,h)
red_root, green_root, blue_root = built_trees[index]

build_matrix_based_on_tree(built_R, red_root)
build_matrix_based_on_tree(built_G, green_root)
build_matrix_based_on_tree(built_B, blue_root)

In [ ]:
built_R = built_R./255
built_G = built_G./255
built_B = built_B./255;

built_R = min.(built_R, 1)
built_R = max.(built_R, 0)

built_G = min.(built_G, 1)
built_G = max.(built_G, 0)

built_B = min.(built_B, 1)
built_B = max.(built_B, 0);

Porównanie czerwonego (R):

In [ ]:
red_filename = @sprintf("bitmaps/red_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1R, built_R, red_filename, whitespace_between=40)
load(red_filename)

Porównanie zielonego (G):

In [ ]:
green_filename = @sprintf("bitmaps/green_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1G, built_G, green_filename, whitespace_between=40)
load(green_filename)

Porówananie niebieskiego (B):

In [ ]:
blue_filename = @sprintf("bitmaps/blue_%d_comparison.png", index)
create_image_comparing_bitmaps(abstract1B, built_B, blue_filename, whitespace_between=40)
load(blue_filename)

Porównanie całości:

In [ ]:
overall_filename = @sprintf("bitmaps/overall_%d_comparison.png", index)
create_image_comparing_RGB_bitmaps(abstract1R, abstract1G, abstract1B, built_R, built_G, built_B, overall_filename, whitespace_between=40)
load(overall_filename)

### Dobranie najlepszych hiperparametrów:

W kompresji macierzy istotne jest nie tylko dobre przybliżenie rzeczywistych danych, lecz również to by te przybliżające dane zajęły mniej miejsca niż oryginalna macierz. Z racji tego jak tworzona jest struktura drzewa kompresji przy niefortunnym doborze hiperparametrów ryzykujemy tych, że liczba danych "skompresowanych" będzie większa niż liczba danych z oryginalenj macierzy. W tej sekcji dokonano próby optymalizacji parametrów algroytmu kompresji.

Ocenę modelu będziemy dokonywać poprzez wyliczenie funkcji jakości, która opiera się na:

* średnim błędzie średniokwadratowym,
* liczbie przechowywanych danych względem oryginalnej macierzy.

To jak istotne powyższe będą w ocenie modeli dyktują kolejno $w_{msq}$, $w_{rs}$:

In [ ]:
w_msq = 1.5
w_rs = 2.5;

Poniżej znajduje się kod do czytania liczby danych w drzewku kompresji:

In [ ]:
function count_all_matrix_data_in_compr_tree(node::CompressionTreeModule.CompressionTreeNode)
    matrix_data_size = 0
    if !isnothing(node.U_matrix)
        matrix_data_size += length(node.U_matrix)
    end
    if !isnothing(node.V_tr_matrix)
        matrix_data_size += length(node.V_tr_matrix)
    end 
    
    if !isnothing(node.left_upper_child)
        matrix_data_size += count_all_matrix_data_in_compr_tree(node.left_upper_child)
    end
    if !isnothing(node.right_upper_child)
        matrix_data_size += count_all_matrix_data_in_compr_tree(node.right_upper_child)
    end
    if !isnothing(node.left_lower_child)
        matrix_data_size += count_all_matrix_data_in_compr_tree(node.left_lower_child)
    end
    if !isnothing(node.right_lower_child)
        matrix_data_size += count_all_matrix_data_in_compr_tree(node.right_lower_child)
    end

    return matrix_data_size
end;

In [ ]:
slight_increase = 1e-2
num_of_egvalsR = size(eigenvaluesR,1) 
num_of_egvalsG = size(eigenvaluesG,1)
num_of_egvalsB = size(eigenvaluesB,1)
gamma_low = 1
gamma_high = 4
model_qualities = [0. for _ in gamma_low:1:gamma_high, _ in 1:1:num_of_egvalsR-1]

image_size = length(abstract1R_255);

In [ ]:
for gamma in gamma_low:1:gamma_high
    for i in 1:1:num_of_egvalsR-1
                
        red_root = create_tree(abstract1R_255, gamma=gamma, eps=eigenvaluesR[i+1]+slight_increase)
        green_root = create_tree(abstract1G_255, gamma=gamma, eps=eigenvaluesG[i+1]+slight_increase)
        blue_root = create_tree(abstract1B_255, gamma=gamma, eps=eigenvaluesB[i+1]+slight_increase)

        red_matrix = zeros(size(abstract1R_255,1), size(abstract1R_255,2))
        green_matrix = zeros(size(abstract1G_255,1), size(abstract1G_255,2))
        blue_matrix = zeros(size(abstract1B_255,1), size(abstract1B_255,2))

        build_matrix_based_on_tree(red_matrix, red_root)
        build_matrix_based_on_tree(green_matrix, green_root)
        build_matrix_based_on_tree(blue_matrix, blue_root)

        red_diffs = abs.(abstract1R_255-red_matrix)
        green_diffs = abs.(abstract1G_255-green_matrix)
        blue_diffs = abs.(abstract1B_255-blue_matrix)

        found_maximum = findmax(red_diffs)[1]
        found_maximum = max(found_maximum, findmax(green_diffs)[1])
        found_maximum = max(found_maximum, findmax(blue_diffs)[1])

        red_diffs ./= found_maximum
        green_diffs ./= found_maximum
        blue_diffs ./= found_maximum

        avg_sq_diff = ((sum(red_diffs.^2)) + (sum(green_diffs.^2)) + (sum(blue_diffs.^2))) / (3*image_size)

        matrix_size_red = count_all_matrix_data_in_compr_tree(red_root)
        matrix_size_green = count_all_matrix_data_in_compr_tree(green_root)
        matrix_size_blue = count_all_matrix_data_in_compr_tree(blue_root)

        overall_relative_size = (matrix_size_red + matrix_size_green + matrix_size_blue) / (6*image_size)

        calc_quality = (w_msq*avg_sq_diff + w_rs*overall_relative_size) / (w_msq + w_rs)

        model_qualities[gamma,i] = calc_quality
                
    end
end